# STLRom Tutorial
First we install and import the library.

In [1]:
import os
import sys
import importlib
# Get the absolute path to the directory containing the local module
stlrom_path = os.path.abspath('../build')

# Insert the local module path at the beginning of sys.path
sys.path.insert(0, stlrom_path)

import stlrom

# Reload the module to ensure it is loaded from the new path
importlib.reload(stlrom)

import stlrom as sr
import numpy as np

In [1]:
#!pip install stlrom # can take a couple of minutes to build initially
import stlrom

## Parsing Formulas
We instantiate an STLDriver object. Its purpose is to parse STL specification read in strings or files.

In [2]:
stl_driver = stlrom.STLDriver()

We need to declare signals, parameters (optional), and formulas. For this we use the `parse_string()` method.

In [3]:
stl_driver.parse_string("signal x, y")       # declaring signals named 'x' and 'y'
stl_driver.parse_string("param p=0.2, tau=5") # declaring parameters named 'p' and 'q'

True

Let's declare formulas. Note that formulas previously defined can be re-used as sub-formulas.

In [4]:
stl_driver.parse_string("mu_x := x[t] > 0 ") # a atomic proposition
stl_driver.parse_string("mu_y := y[t] < p ") # another atomic proposition with a parameter
stl_driver.parse_string("mu := mu_x and mu_y") # a compound proposition
stl_driver.parse_string("phi := ev_[0, tau] mu") # a temporal proposition with parameter tau

True

## Adding Timed Samples Data
We can now add data. We do it by adding time valued samples of the form [t, x, y]. Be careful to respect the order of signals declaration.

In [5]:
stl_driver.add_sample([0, 2., 3.])     # at time 0, x=2, y=3
stl_driver.add_sample([0.5, -1., -4.]) # at time 0.5, x=-1, y=-4
stl_driver.add_sample([3., 3., -5.])    # etc.
print(stl_driver.data)

[[0.0, 2.0, 3.0], [0.5, -1.0, -4.0], [3.0, 3.0, -5.0]]


## Eval Formula Satisfaction

We are now ready to evaluate formulas. For this we use the get_online_rob() method.

In [6]:
rob_mu_x  = stl_driver.get_online_rob("mu_x")
rob_mu_y  = stl_driver.get_online_rob("mu_y")
rob_mu    = stl_driver.get_online_rob("mu")
rob_phi = stl_driver.get_online_rob("phi")

print("rob_mu_x = ", rob_mu_x)
print("rob_mu_y = ", rob_mu_y)
print("rob_mu = ", rob_mu)
print("rob_phi = ", rob_phi)

rob_mu_x =  [2.0, 2.0, 2.0]
rob_mu_y =  [-2.8, -2.8, -2.8]
rob_mu =  [-2.8, -2.8, -2.8]
rob_phi =  [3.0, 3.0, 10000.0]


`get_online_rob("phi")` returns three values: `[rob_estimate, rob_low, rob_up]`. They have the following characteristics:
- if the `phi` can be fully evaluated by the data, then all three values are equal to the actual robust satisfaction `rob` of the formula. On the example above, this is the case for atomic propositions, which are evaluated at time 0 by default.
- Otherwise, `[rob_estimate, rob_low, rob_up]` are respectively an estimate, a lower bound and an upper bound of `rob`. In the example above, `phi` cannot be evaluated because we have data up to time 3 and the formula requires data up to time `tau`, which is 5. However, stlrom computed that rob is between 3 and 10000, which means that we can already assert that the formula is satisfied (rob is positive.)

Note that 10000 is an arbitrary large number that can be changed using the set_BigM() method.

In [7]:
s = stlrom.Signal().set_BigM(5000) # set the BigM value globally to 5000

In [8]:
rob_phi = stl_driver.get_online_rob("phi")
print(rob_phi)

[3.0, 3.0, 5000.0]


## Changing Parameters
Parameters can be changed using set_param().


In [9]:
stl_driver.set_param("tau", 3.) # change the value of the parameter tau, from 5 to 3
rob_phi = stl_driver.get_online_rob("phi")
print(rob_phi)

[3.0, 3.0, 3.0]


## Evaluating Formulas From Time Greater Than 0
Note that the robustness value of phi is now fully computed, since we reduced the time bound of the eventually operator. `get_online_rob()` can be called at a time different than 0.

In [10]:
rob_mu_at_1 = stl_driver.get_online_rob("mu_x", 1)
print("rob_mu_at_1:", rob_mu_at_1)

rob_phi_at_1 = stl_driver.get_online_rob("phi", 1)
print("rob_phi_at_1:", rob_phi_at_1)


rob_mu_at_1: [-0.19999999999999996, -0.19999999999999996, -0.19999999999999996]
rob_phi_at_1: [3.0, 3.0, 5000.0]


Note that data is interpolated linearly between samples. Also from time 1, the horizon of the eventually operator again goes beyond what we have in the data. The interpolation method can be changed to be piecewise constant.

In [11]:
stl_driver.get_interpol()

'LINEAR'

For now only 'LINEAR' and 'PREVIOUS' interpolation are available.

In [20]:
stl_driver.set_interpol('PREVIOUS')


In [21]:
stl_driver.get_interpol()

'PREVIOUS'

In [22]:
rob_mu_at_1 = stl_driver.get_online_rob("mu_x", 1)
print("rob_mu_at_1:", rob_mu_at_1)


rob_mu_at_1: [-1.0, -1.0, -1.0]


## STLMonitor

A STLDriver object can be used to create standalone monitors as STLMonitor object. They inherit data, parameters, options, etc from the STLDriver object, but can only monitor one formula and cannot parse any more.

In [23]:
phi = stl_driver.get_monitor('phi')

`phi` gets a copy of `stl_driver` data. It can now get more samples and monitor them independantly.

In [24]:
#phi.add_sample([4,3,5])
phi.eval_rob()

-1.0

In [25]:
phi.get_interpol()

'PREVIOUS'

In [27]:
phi.set_interpol('LINEAR')
phi.eval_rob()

interpol:LINEAR
lin


3.0

In [28]:
print(phi.data)

[[0.0, 2.0, 3.0], [0.5, -1.0, -4.0], [3.0, 3.0, -5.0]]


In [17]:
print(phi)

Signal Names: x, y
Parameters: p: 0.2, tau: 3
Formula: ev_[0,tau] (x[t] > 0 and y[t] < p) 



In [29]:
phi.set_interpol('LINEAR')

interpol:LINEAR
lin


In [30]:
phi.set_param('tau',3)
phi.eval_rob()

3.0